<a href="https://colab.research.google.com/github/CjRax/Pythonworkspace/blob/master/cmfrec_movielens_sideinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative filtering with side information

** *
This IPython notebook illustrates the usage of the [cmfrec](https://github.com/david-cortes/cmfrec) Python package for building recommender systems through different matrix factorization models with or without using information about user and item attributes – for more details see the references at the bottom.

The example uses the [MovieLens-1M data](https://grouplens.org/datasets/movielens/1m/) which consists of ratings from users about movies + user demographic information, plus the [movie tag genome](https://grouplens.org/datasets/movielens/latest/). Note however that, for implicit-feedback datasets (e.g. item purchases), it's recommended to use different models than the ones shown here (see [documentation](http://ctpfrec.readthedocs.io/en/latest/) for details about models in the package aimed at implicit-feedback data).

**Small note: if the TOC here is not clickable or the math symbols don't show properly, try visualizing this same notebook from nbviewer following [this link](http://nbviewer.jupyter.org/github/david-cortes/cmfrec/blob/master/example/cmfrec_movielens_sideinfo.ipynb).**
## Sections


[1. Loading the data](#p1)

[2. Fitting recommender models](#p2)

[3. Examining top-N recommended lists](#p3)

[4. Tuning model parameters](#p4)

[5. Recommendations for new users](#p5)

[6. Evaluating models](#p6)

[7. References](#p7)
** *

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
java8_location= '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['JAVA_HOME'] = java8_location
main_location = "/content/gdrive/My Drive/Colab Notebooks/Collaborative filtering with side information"
os.chdir(main_location)

In [5]:
!pip install cmfrec

     |████████████████████████████████| 163kB 7.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Processing /root/.cache/pip/wheels/f3/2a/54/a839ba95bc8518ba293d78b4b4ec6257482d845079e2447cae/findblas-0.1.8.1-cp36-none-any.whl
  Created wheel for cmfrec: filename=cmfrec-1.0.0-cp36-cp36m-linux_x86_64.whl size=2932150 sha256=caa41c44e6f5e3e2fc0a1bd847c5c7a3d4f128dbedd71a6b921eae9257e62bd0
  Stored in directory: /root/.cache/pip/wheels/25/ec/05/8a323008953b5708a0be0e023f55b8a7cd448c51a13d7bdf1e
Successfully built cmfrec


<a id="p1"></a>
## 1. Loading the data

This section uses pre-processed data from the MovieLens datasets joined with external zip codes databases. The script for  processing and cleaning the data can be found in another notebook [here](http://nbviewer.jupyter.org/github/david-cortes/cmfrec/blob/master/example/load_data.ipynb).

In [0]:
import numpy as np, pandas as pd, pickle

ratings = pickle.load(open("ratings.p", "rb"))
item_sideinfo_pca = pickle.load(open("item_sideinfo_pca.p", "rb"))
user_side_info = pickle.load(open("user_side_info.p", "rb"))
movie_id_to_title = pickle.load(open("movie_id_to_title.p", "rb"))

### Ratings data

In [7]:
ratings.head()

,UserId,ItemId,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


### Item attributes (reduced through PCA)

In [8]:
item_sideinfo_pca.head()

,ItemId,pc0,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11,pc12,pc13,pc14,pc15,pc16,pc17,pc18,pc19,pc20,pc21,pc22,pc23,pc24,pc25,pc26,pc27,pc28,pc29,pc30,pc31,pc32,pc33,pc34,pc35,pc36,pc37,pc38,pc39,pc40,pc41,pc42,pc43,pc44,pc45,pc46,pc47,pc48,pc49
0,1,1.192433,2.034965,2.679781,1.154823,0.715302,0.982528,1.251208,-0.792800,1.605826,-0.737271,-0.183611,0.284967,-0.904565,-0.518614,0.089596,0.651764,0.679378,0.728897,-0.009248,-0.165799,0.067822,0.216426,-0.010873,0.279555,0.772343,-0.165439,-0.340126,0.313842,-0.430782,0.038745,-0.260901,-0.020728,0.107546,-0.192297,-0.032513,-0.296049,0.349338,-0.065485,-0.237691,-0.276438,-0.310903,-0.086192,-0.022227,-0.225551,0.208888,0.053772,-0.235847,-0.218742,-0.044116,-0.159722
1,2,-1.333200,1.719346,1.383137,0.788332,-0.487431,0.376546,0.803104,-0.606602,0.914494,-0.430699,-0.404877,0.347346,-0.157892,-0.509014,0.518503,-0.374896,0.229185,0.147371,-0.269713,0.721060,-0.393251,-0.173606,-0.451843,-0.337946,0.202661,-0.132015,-0.269896,-0.125917,-0.309087,-0.675771,0.054510,-0.439607,-0.005626,-0.224508,0.177771,-0.088804,-0.165457,0.091724,0.290934,-0.103507,0.275732,-0.295407,0.049265,-0.033086,0.295205,0.353795,0.271795,0.152945,0.118240,0.049958
2,3,-1.363421,-0.034093,0.528633,-0.312122,0.468820,0.164593,0.021909,0.161554,-0.231992,0.067063,0.070401,-0.545160,-0.236044,0.388355,0.191815,0.126100,0.237735,0.403455,-0.237825,0.004958,0.797680,0.296292,-0.446395,0.235011,-0.300256,0.081383,0.611506,0.157810,0.246292,-0.297811,-0.279625,0.029209,0.310131,0.331211,-0.295085,0.181023,-0.008465,0.058480,0.015182,-0.079272,0.214418,-0.096803,-0.279147,-0.042245,0.064378,0.239892,-0.169373,-0.097961,0.164606,-0.135193
3,4,-1.238094,-1.014399,0.790394,-0.296004,-0.095043,-0.052266,-0.180244,-0.768811,-0.400559,0.170457,0.144012,-0.260407,-0.190370,-0.442000,0.131109,0.141005,-0.405538,-0.453870,-0.316645,0.013256,0.335646,0.061231,0.118042,-0.011508,-0.123344,-0.144591,0.158255,0.000388,-0.055361,-0.076103,-0.155165,0.156729,0.001212,-0.052611,0.046802,-0.093289,0.187653,-0.202027,0.044542,-0.063037,0.073693,0.040068,-0.222469,-0.447134,0.262161,-0.346505,-0.197113,-0.206347,0.456150,0.442459
4,5,-1.613220,-0.280142,1.119149,-0.130238,0.397091,0.187158,0.108864,-0.273748,-0.260166,-0.336405,0.135059,-0.848177,-0.483407,0.267419,0.604689,0.298647,0.347358,0.345831,-0.172015,-0.034484,0.447468,0.231412,-0.169929,-0.023544,-0.911886,0.267502,0.332914,0.227506,0.400202,-0.709007,-0.372312,0.084479,0.375738,0.267540,-0.175196,0.110716,0.037989,-0.065917,0.187277,-0.249537,0.115972,-0.114713,-0.210003,-0.474771,-0.030351,0.121522,-0.225187,-0.289037,0.055255,0.047612


### User attributes (one-hot encoded)

In [9]:
user_side_info.head()

,UserId,Gender_F,Gender_M,Age_1,Age_18,Age_25,Age_35,Age_45,Age_50,Age_56,"Occupation_""other"" or not specified",Occupation_K-12 student,Occupation_academic/educator,Occupation_artist,Occupation_clerical/admin,Occupation_college/grad student,Occupation_customer service,Occupation_doctor/health care,Occupation_executive/managerial,Occupation_farmer,Occupation_homemaker,Occupation_lawyer,Occupation_programmer,Occupation_retired,Occupation_sales/marketing,Occupation_scientist,Occupation_self-employed,Occupation_technician/engineer,Occupation_tradesman/craftsman,Occupation_unemployed,Occupation_writer,Region_Middle Atlantic,Region_Midwest,Region_New England,Region_South,Region_Southwest,Region_UnknownOrNonUS,Region_UsOther,Region_West
0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,5,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0


<a id="p2"></a>
## 2. Fitting recommender models

This section fits different recommendation models and then compares the recommendations produced by them.

### 2.1 Cassic model

Usual low-rank matrix factorization model with no user/item attributes:
$$
\mathbf{X} \approx \mathbf{A} \mathbf{B}^T + \mu + \mathbf{b}_A + \mathbf{b}_B
$$
Where
* $\mathbf{X}$ is the ratings matrix, in which users are rows, items are columns, and the entries denote the ratings.
* $\mathbf{A}$ is the user-factors matrix.
* $\mathbf{B}$ is the item-factors matrix.
* $\mu$ is the average rating.
* $\mathbf{b}_A$ are user-specific biases (row vector).
* $\mathbf{b}_B$ are item-specific biases (column vector).

(For more details see references at the bottom)

In [10]:
%%time
from cmfrec import CMF_explicit

model_no_sideinfo = CMF_explicit(method="als", k=40, lambda_=1e+1)
model_no_sideinfo.fit(ratings)

CPU times: user 16.8 s, sys: 32.6 ms, total: 16.9 s
Wall time: 9.11 s


### 2.2 Collective model

The collective matrix factorization model extends the earlier model by making the user and item factor matrices also be able to make low-rank approximate factorizations of the user and item attributes:
$$
\mathbf{X} \approx \mathbf{A} \mathbf{B}^T + \mu + \mathbf{b}_A + \mathbf{b}_B
,\:\:\:\:
\mathbf{U} \approx \mathbf{A} \mathbf{C}^T + \mathbf{\mu}_U
,\:\:\:\: \mathbf{I} \approx \mathbf{B} \mathbf{D}^T + \mathbf{\mu}_I
$$

Where
* $\mathbf{U}$ is the user attributes matrix, in which users are rows and attributes are columns.
* $\mathbf{I}$ is the item attributes matrix, in which items are rows and attributes are columns.
* $\mathbf{\mu}_U$ are the column means for the user attributes (column vector).
* $\mathbf{\mu}_I$ are the columns means for the item attributes (column vector).
* $\mathbf{C}$ and $\mathbf{D}$ are attribute-factor matrices (also model parameters).

**In addition**, this package can also apply sigmoid transformations on the attribute columns which are binary. Note that this requires a different optimization approach which is slower than the ALS (alternating least-squares) method used here.

In [11]:
%%time
model_with_sideinfo = CMF_explicit(method="als", k=40, lambda_=1e+1, w_main=0.5, w_user=0.25, w_item=0.25)
model_with_sideinfo.fit(X=ratings, U=user_side_info, I=item_sideinfo_pca)

### for the sigmoid transformations:
# model_with_sideinfo = CMF_explicit(method="lbfgs", maxiter=0, k=40, lambda_=1e+1, w_main=0.5, w_user=0.25, w_item=0.25)
# model_with_sideinfo.fit(X=ratings, U_bin=user_side_info, I=item_sideinfo_pca)

CPU times: user 19 s, sys: 48.6 ms, total: 19 s
Wall time: 9.92 s


_(Note that, since the side info has variables in a different scale, even though the weights sum up to 1, it's still not the same as the earlier model w.r.t. the regularization parameter - this type of model requires more hyperparameter tuning too.)_

### 2.3 Content-based model

This is a model in which the factorizing matrices are constrained to be linear combinations of the user and item attributes, thereby making the recommendations based entirely on side information, with no free parameters for specific users or items:
$$
\mathbf{X} \approx (\mathbf{U} \mathbf{C}) (\mathbf{I} \mathbf{D})^T + \mu
$$

_(Note that the movie attributes are not available for all the movies with ratings)_

In [0]:
%%time
from cmfrec import ContentBased

model_content_based = ContentBased(k=40, maxiter=0, user_bias=False, item_bias=False)
model_content_based.fit(X=ratings.loc[ratings.ItemId.isin(item_sideinfo_pca.ItemId)],
                        U=user_side_info,
                        I=item_sideinfo_pca.loc[item_sideinfo_pca.ItemId.isin(ratings.ItemId)])

CPU times: user 39min 3s, sys: 20.2 s, total: 39min 23s
Wall time: 2min 30s


Content-based factorization model
(explicit-feedback)


In [12]:
%%time
from cmfrec import ContentBased

model_content_based = ContentBased(k=40, maxiter=0, user_bias=False, item_bias=False)
model_content_based.fit(X=ratings.loc[ratings.ItemId.isin(item_sideinfo_pca.ItemId)],
                        U=user_side_info,
                        I=item_sideinfo_pca.loc[item_sideinfo_pca.ItemId.isin(ratings.ItemId)])

CPU times: user 16min 18s, sys: 1.26 s, total: 16min 19s
Wall time: 8min 17s


### 2.4 Non-personalized model

This is an intercepts-only version of the classical model, which estimates one parameter per user and one parameter per item, and as such produces a simple rank of the items based on those parameters. It is intended for comparison purposes and can be helpful to check that the recommendations for different users are having some variability (e.g. setting too large regularization values will tend to make all personalzied recommended lists similar to each other).

In [13]:
%%time
from cmfrec import MostPopular

model_non_personalized = MostPopular(user_bias=True, implicit=False)
model_non_personalized.fit(ratings)

CPU times: user 82.1 ms, sys: 2.99 ms, total: 85.1 ms
Wall time: 92.6 ms


<a id="p3"></a>
## 3. Examining top-N recommended lists

This section will examine what would each model recommend to the user with ID 948.

This is the demographic information for the user:

In [14]:
user_side_info.loc[user_side_info.UserId == 948].T.where(lambda x: x > 0).dropna()

,947
UserId,948.0
Gender_M,1.0
Age_56,1.0
Occupation_programmer,1.0
Region_Midwest,1.0


These are the highest-rated movies from the user:

In [15]:
ratings\
    .loc[ratings.UserId == 948]\
    .sort_values("Rating", ascending=False)\
    .assign(Movie=lambda x: x.ItemId.map(movie_id_to_title))\
    .head(10)

,UserId,ItemId,Rating,Movie
146721,948,3789,5,"Pawnbroker, The (1965)"
146889,948,2665,5,Earth Vs. the Flying Saucers (1956)
146871,948,2640,5,Superman (1978)
146872,948,2641,5,Superman II (1980)
147105,948,2761,5,"Iron Giant, The (1999)"
146875,948,2644,5,Dracula (1931)
146878,948,2648,5,Frankenstein (1931)
147097,948,1019,5,"20,000 Leagues Under the Sea (1954)"
146881,948,2657,5,"Rocky Horror Picture Show, The (1975)"
146884,948,2660,5,"Thing From Another World, The (1951)"


These are the lowest-rated movies from the user:

In [16]:
ratings\
    .loc[ratings.UserId == 948]\
    .sort_values("Rating", ascending=True)\
    .assign(Movie=lambda x: x.ItemId.map(movie_id_to_title))\
    .head(10)

,UserId,ItemId,Rating,Movie
147237,948,1247,1,"Graduate, The (1967)"
147173,948,70,1,From Dusk Till Dawn (1996)
146768,948,748,1,"Arrival, The (1996)"
147135,948,45,1,To Die For (1995)
146812,948,780,1,Independence Day (ID4) (1996)
146813,948,788,1,"Nutty Professor, The (1996)"
146814,948,3201,1,Five Easy Pieces (1970)
147118,948,356,1,Forrest Gump (1994)
146821,948,3070,1,Adventures of Buckaroo Bonzai Across the 8th D...
146822,948,1617,1,L.A. Confidential (1997)


Now producing recommendations from each model:

In [0]:
### Will exclude already-seen movies
exclude = ratings.ItemId.loc[ratings.UserId == 948]
exclude_cb = exclude.loc[exclude.isin(item_sideinfo_pca.ItemId)]

### Recommended lists with those excluded
recommended_non_personalized = model_non_personalized.topN(user=948, n=10, exclude=exclude)
recommended_no_side_info = model_no_sideinfo.topN(user=948, n=10, exclude=exclude)
recommended_with_side_info = model_with_sideinfo.topN(user=948, n=10, exclude=exclude)
recommended_content_based = model_content_based.topN(user=948, n=10, exclude=exclude_cb)

In [18]:
recommended_non_personalized

array([2019,  318, 2905,  745, 1148, 1212, 3435,  923,  720, 3307])

A handy function to print top-N recommended lists with associated information:

In [19]:
from collections import defaultdict

# aggregate statistics
avg_movie_rating = defaultdict(lambda: 0)
num_ratings_per_movie = defaultdict(lambda: 0)
for i in ratings.groupby('ItemId')['Rating'].mean().to_frame().itertuples():
    avg_movie_rating[i.Index] = i.Rating
for i in ratings.groupby('ItemId')['Rating'].agg(lambda x: len(tuple(x))).to_frame().itertuples():
    num_ratings_per_movie[i.Index] = i.Rating

# function to print recommended lists more nicely
def print_reclist(reclist):
    list_w_info = [str(m + 1) + ") - " + movie_id_to_title[reclist[m]] +\
        " - Average Rating: " + str(np.round(avg_movie_rating[reclist[m]], 2))+\
        " - Number of ratings: " + str(num_ratings_per_movie[reclist[m]])\
                   for m in range(len(reclist))]
    print("\n".join(list_w_info))
    
print("Recommended from non-personalized model")
print_reclist(recommended_non_personalized)
print("----------------")
print("Recommended from ratings-only model")
print_reclist(recommended_no_side_info)
print("----------------")
print("Recommended from attributes-only model")
print_reclist(recommended_content_based)
print("----------------")
print("Recommended from hybrid model")
print_reclist(recommended_with_side_info)

Recommended from non-personalized model
1) - Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) - Average Rating: 4.56 - Number of ratings: 628
2) - Shawshank Redemption, The (1994) - Average Rating: 4.55 - Number of ratings: 2227
3) - Sanjuro (1962) - Average Rating: 4.61 - Number of ratings: 69
4) - Close Shave, A (1995) - Average Rating: 4.52 - Number of ratings: 657
5) - Wrong Trousers, The (1993) - Average Rating: 4.51 - Number of ratings: 882
6) - Third Man, The (1949) - Average Rating: 4.45 - Number of ratings: 480
7) - Double Indemnity (1944) - Average Rating: 4.42 - Number of ratings: 551
8) - Citizen Kane (1941) - Average Rating: 4.39 - Number of ratings: 1116
9) - Wallace & Gromit: The Best of Aardman Animation (1996) - Average Rating: 4.43 - Number of ratings: 438
10) - City Lights (1931) - Average Rating: 4.39 - Number of ratings: 271
----------------
Recommended from ratings-only model
1) - Babe (1995) - Average Rating: 3.89 - Number of ratings: 1751
2) -

(As can be seen, the personalized recommendations tend to recommend very old movies, which is what this user seems to rate highly, with no overlap with the non-personalized recommendations).

<a id="p4"></a>
## 4. Tuning model parameters

The models here offer many tuneable parameters which can be tweaked in order to alter the recommended lists in some way. For example, setting a low regularization to the item biases will tend to favor movies with a high average rating regardless of the number of ratings, while setting a high regularization for the factorizing matrices will tend to produce the same recommendations for all users.

In [20]:
### Less personalized (underfitted)
reclist = \
    CMF_explicit(lambda_=[1e+3, 1e+1, 1e+2, 1e+2, 1e+2, 1e+2])\
        .fit(ratings)\
        .topN(user=948, n=10, exclude=exclude)
print_reclist(reclist)

1) - Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) - Average Rating: 4.56 - Number of ratings: 628
2) - Shawshank Redemption, The (1994) - Average Rating: 4.55 - Number of ratings: 2227
3) - Close Shave, A (1995) - Average Rating: 4.52 - Number of ratings: 657
4) - Wrong Trousers, The (1993) - Average Rating: 4.51 - Number of ratings: 882
5) - Sanjuro (1962) - Average Rating: 4.61 - Number of ratings: 69
6) - Third Man, The (1949) - Average Rating: 4.45 - Number of ratings: 480
7) - Double Indemnity (1944) - Average Rating: 4.42 - Number of ratings: 551
8) - Wallace & Gromit: The Best of Aardman Animation (1996) - Average Rating: 4.43 - Number of ratings: 438
9) - Citizen Kane (1941) - Average Rating: 4.39 - Number of ratings: 1116
10) - City Lights (1931) - Average Rating: 4.39 - Number of ratings: 271


In [21]:
### More personalized (overfitted)
reclist = \
    CMF_explicit(lambda_=[0., 1e+3, 1e-1, 1e-1, 1e-1, 1e-1])\
        .fit(ratings)\
        .topN(user=948, n=10, exclude=exclude)
print_reclist(reclist)

1) - White Man's Burden (1995) - Average Rating: 2.46 - Number of ratings: 52
2) - Funny Bones (1995) - Average Rating: 3.41 - Number of ratings: 39
3) - Dangerous Beauty (1998) - Average Rating: 3.65 - Number of ratings: 100
4) - Get Carter (1971) - Average Rating: 3.47 - Number of ratings: 55
5) - Fandango (1985) - Average Rating: 3.67 - Number of ratings: 66
6) - World of Apu, The (Apur Sansar) (1959) - Average Rating: 4.41 - Number of ratings: 56
7) - House of Frankenstein (1944) - Average Rating: 3.19 - Number of ratings: 52
8) - Adventures of Pinocchio, The (1996) - Average Rating: 2.91 - Number of ratings: 54
9) - Jude (1996) - Average Rating: 3.54 - Number of ratings: 50
10) - Duel in the Sun (1946) - Average Rating: 3.56 - Number of ratings: 72


The collective model can also have variations such as weighting each factorization differently, or setting components (factors) that are not to be shared between factorizations (not shown).

In [22]:
### More oriented towards content-based than towards collaborative-filtering
reclist = \
    CMF_explicit(k=40, w_main=0.5, w_item=3., w_user=5., lambda_=1e+1)\
        .fit(ratings, U=user_side_info, I=item_sideinfo_pca)\
        .topN(user=948, n=10, exclude=exclude)
print_reclist(reclist)

1) - Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922) - Average Rating: 3.99 - Number of ratings: 238
2) - Singin' in the Rain (1952) - Average Rating: 4.28 - Number of ratings: 751
3) - It Happened One Night (1934) - Average Rating: 4.28 - Number of ratings: 374
4) - It's a Wonderful Life (1946) - Average Rating: 4.3 - Number of ratings: 729
5) - Invasion of the Body Snatchers (1956) - Average Rating: 3.91 - Number of ratings: 628
6) - Arsenic and Old Lace (1944) - Average Rating: 4.17 - Number of ratings: 672
7) - Miracle on 34th Street (1947) - Average Rating: 3.96 - Number of ratings: 380
8) - Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) - Average Rating: 4.56 - Number of ratings: 628
9) - Bride of Frankenstein (1935) - Average Rating: 3.91 - Number of ratings: 216
10) - Gold Rush, The (1925) - Average Rating: 4.19 - Number of ratings: 275


<a id="p5"></a>
## 5. Recommendations for new users

Models can also be used to make recommendations for new users based on ratings and/or side information.

_(Be aware that, due to the nature of computer floating point aithmetic, there might be some slight discrepancies between the results from `topN` and `topN_warm`)_

In [23]:
print_reclist(model_with_sideinfo.topN_warm(X_col=ratings.ItemId.loc[ratings.UserId == 948],
                                            X_val=ratings.Rating.loc[ratings.UserId == 948],
                                            exclude=exclude))

1) - Arsenic and Old Lace (1944) - Average Rating: 4.17 - Number of ratings: 672
2) - Babe (1995) - Average Rating: 3.89 - Number of ratings: 1751
3) - Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922) - Average Rating: 3.99 - Number of ratings: 238
4) - Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) - Average Rating: 4.56 - Number of ratings: 628
5) - Night of the Living Dead (1968) - Average Rating: 3.67 - Number of ratings: 715
6) - Gold Rush, The (1925) - Average Rating: 4.19 - Number of ratings: 275
7) - Christmas Carol, A (1938) - Average Rating: 3.99 - Number of ratings: 194
8) - Singin' in the Rain (1952) - Average Rating: 4.28 - Number of ratings: 751
9) - Birds, The (1963) - Average Rating: 3.88 - Number of ratings: 733
10) - Mr. Smith Goes to Washington (1939) - Average Rating: 4.24 - Number of ratings: 383


In [24]:
print_reclist(model_with_sideinfo.topN_warm(X_col=ratings.ItemId.loc[ratings.UserId == 948],
                                            X_val=ratings.Rating.loc[ratings.UserId == 948],
                                            U=user_side_info.loc[user_side_info.UserId == 948],
                                            exclude=exclude))

1) - Arsenic and Old Lace (1944) - Average Rating: 4.17 - Number of ratings: 672
2) - Babe (1995) - Average Rating: 3.89 - Number of ratings: 1751
3) - Nosferatu (Nosferatu, eine Symphonie des Grauens) (1922) - Average Rating: 3.99 - Number of ratings: 238
4) - Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) - Average Rating: 4.56 - Number of ratings: 628
5) - Night of the Living Dead (1968) - Average Rating: 3.67 - Number of ratings: 715
6) - Gold Rush, The (1925) - Average Rating: 4.19 - Number of ratings: 275
7) - Christmas Carol, A (1938) - Average Rating: 3.99 - Number of ratings: 194
8) - Singin' in the Rain (1952) - Average Rating: 4.28 - Number of ratings: 751
9) - Birds, The (1963) - Average Rating: 3.88 - Number of ratings: 733
10) - Mr. Smith Goes to Washington (1939) - Average Rating: 4.24 - Number of ratings: 383


In [25]:
print_reclist(model_with_sideinfo.topN_cold(U=user_side_info.loc[user_side_info.UserId == 948].drop("UserId", axis=1),
                                            exclude=exclude))

1) - Shawshank Redemption, The (1994) - Average Rating: 4.55 - Number of ratings: 2227
2) - Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) - Average Rating: 4.56 - Number of ratings: 628
3) - Wrong Trousers, The (1993) - Average Rating: 4.51 - Number of ratings: 882
4) - Close Shave, A (1995) - Average Rating: 4.52 - Number of ratings: 657
5) - Sanjuro (1962) - Average Rating: 4.61 - Number of ratings: 69
6) - Wallace & Gromit: The Best of Aardman Animation (1996) - Average Rating: 4.43 - Number of ratings: 438
7) - Life Is Beautiful (La Vita � bella) (1997) - Average Rating: 4.33 - Number of ratings: 1152
8) - Double Indemnity (1944) - Average Rating: 4.42 - Number of ratings: 551
9) - Grand Day Out, A (1992) - Average Rating: 4.36 - Number of ratings: 473
10) - Third Man, The (1949) - Average Rating: 4.45 - Number of ratings: 480


This last one is very similar to the non-personalized recommended list - that is, the user side information had very little leverage in the model, at least for that user - in this regard, the content-based model tends to be better at cold-start recommendations:

In [26]:
print_reclist(model_content_based.topN_cold(U=user_side_info.loc[user_side_info.UserId == 948].drop("UserId", axis=1),
                                            exclude=exclude_cb))

1) - Shawshank Redemption, The (1994) - Average Rating: 4.55 - Number of ratings: 2227
2) - Third Man, The (1949) - Average Rating: 4.45 - Number of ratings: 480
3) - City Lights (1931) - Average Rating: 4.39 - Number of ratings: 271
4) - Jean de Florette (1986) - Average Rating: 4.32 - Number of ratings: 216
5) - It Happened One Night (1934) - Average Rating: 4.28 - Number of ratings: 374
6) - Central Station (Central do Brasil) (1998) - Average Rating: 4.28 - Number of ratings: 215
7) - Best Years of Our Lives, The (1946) - Average Rating: 4.12 - Number of ratings: 236
8) - Man Who Would Be King, The (1975) - Average Rating: 4.13 - Number of ratings: 310
9) - Good Will Hunting (1997) - Average Rating: 4.18 - Number of ratings: 1548
10) - Double Indemnity (1944) - Average Rating: 4.42 - Number of ratings: 551


_(For this use-case, would also be better to add item biases to the content-based model though)_

<a id="p6"></a>
## 6. Evaluating models

This section shows usage of the `predict` family of functions for getting the predicted  rating for a given user and item, in order to calculate evaluation metrics such as RMSE and tune model parameters.

**Note that, while widely used in earlier literature, RMSE might not provide a good overview of the ranking of items (which is what matters for recommendations), and it's recommended to also evaluate other metrics such as P@K, correlations, etc.**

**Also be aware that there is a different class `CMF_implicit` which might perform better at implicit-feedback matrics such as P@K.**

When making recommendations, there's quite a difference between making predictions based on ratings data or based on side information alone. In this regard, one can classify prediction types into 4 types:
1. Predictions for users and items which were both in the training data.
2. Predictions for users which were in the training data and items which were not in the training data.
3. Predictions for users which were not in the training data and items which were in the training data.
4. Predictions for users and items, of which neither were in the training data.

(One could sub-divide further according to users/items which were present in the training data with only ratings or with only side information, but this notebook will not go into that level of detail)

The classic model is only able to make predictions for the first case, while the collective model can leverage the side information in order to make predictions for (2) and (3). In theory, it could also do (4), but this is not recommended and the API does not provide such functionality.

The content-based model, on the other hand, is an ideal approach for case (4). The package also provides a different model (the "offsets" model - see references at the bottom) aimed at improving cases (2) and (3) when there is side information about only user or only about items at the expense of case (1), but such models are not shown in this notebook.

** *
Producing a training and test set split of the ratings and side information:

In [27]:
from sklearn.model_selection import train_test_split

users_train, users_test = train_test_split(ratings.UserId.unique(), test_size=0.2, random_state=1)
items_train, items_test = train_test_split(ratings.ItemId.unique(), test_size=0.2, random_state=2)

ratings_train, ratings_test1 = train_test_split(ratings.loc[ratings.UserId.isin(users_train) &
                                                            ratings.ItemId.isin(items_train)],
                                                test_size=0.2, random_state=123)
users_train = ratings_train.UserId.unique()
items_train = ratings_train.ItemId.unique()
ratings_test1 = ratings_test1.loc[ratings_test1.UserId.isin(users_train) &
                                  ratings_test1.ItemId.isin(items_train)]

user_attr_train = user_side_info.loc[user_side_info.UserId.isin(users_train)]
item_attr_train = item_sideinfo_pca.loc[item_sideinfo_pca.ItemId.isin(items_train)]

ratings_test2 = ratings.loc[ratings.UserId.isin(users_train) &
                            ~ratings.ItemId.isin(items_train) &
                            ratings.ItemId.isin(item_sideinfo_pca.ItemId)]
ratings_test3 = ratings.loc[~ratings.UserId.isin(users_train) &
                            ratings.ItemId.isin(items_train) &
                            ratings.UserId.isin(user_side_info.UserId) &
                            ratings.ItemId.isin(item_sideinfo_pca.ItemId)]
ratings_test4 = ratings.loc[~ratings.UserId.isin(users_train) &
                            ~ratings.ItemId.isin(items_train) &
                            ratings.UserId.isin(user_side_info.UserId) &
                            ratings.ItemId.isin(item_sideinfo_pca.ItemId)]


print("Number of ratings in training data: %d" % ratings_train.shape[0])
print("Number of ratings in test data type (1): %d" % ratings_test1.shape[0])
print("Number of ratings in test data type (2): %d" % ratings_test2.shape[0])
print("Number of ratings in test data type (3): %d" % ratings_test3.shape[0])
print("Number of ratings in test data type (4): %d" % ratings_test4.shape[0])

Number of ratings in training data: 512972
Number of ratings in test data type (1): 128221
Number of ratings in test data type (2): 153128
Number of ratings in test data type (3): 138904
Number of ratings in test data type (4): 36450


In [0]:
### Handy usage of Pandas indexing
user_attr_test = user_side_info.set_index("UserId")
item_attr_test = item_sideinfo_pca.set_index("ItemId")

Re-fitting earlier models to the training subset of the earlier data:

In [0]:
m_classic = CMF_explicit(k=40)\
                .fit(ratings_train)
m_collective = CMF_explicit(k=40, w_main=0.5, w_user=0.5, w_item=0.5)\
                .fit(X=ratings_train,
                     U=user_attr_train,
                     I=item_attr_train)
m_contentbased = ContentBased(k=40, user_bias=False, item_bias=False)\
                .fit(X=ratings_train.loc[ratings_train.UserId.isin(user_attr_train.UserId) &
                                         ratings_train.ItemId.isin(item_attr_train.ItemId)],
                     U=user_attr_train,
                     I=item_attr_train)
m_mostpopular = MostPopular(user_bias=True)\
                .fit(X=ratings_train)

RMSE for users and items which were both in the training data:

In [30]:
from sklearn.metrics import mean_squared_error

pred_contetbased = m_mostpopular.predict(ratings_test1.UserId, ratings_test1.ItemId)
print("RMSE type 1 non-personalized model: %.3f [rho: %.3f]" %
      (mean_squared_error(ratings_test1.Rating,
                          pred_contetbased,
                          squared=True),
      np.corrcoef(ratings_test1.Rating, pred_contetbased)[0,1]))

pred_ratingsonly = m_classic.predict(ratings_test1.UserId, ratings_test1.ItemId)
print("RMSE type 1 ratings-only model: %.3f [rho: %.3f]" %
      (mean_squared_error(ratings_test1.Rating,
                          pred_ratingsonly,
                          squared=True),
       np.corrcoef(ratings_test1.Rating, pred_ratingsonly)[0,1]))

pred_hybrid = m_collective.predict(ratings_test1.UserId, ratings_test1.ItemId)
print("RMSE type 1 hybrid model: %.3f [rho: %.3f]" %
      (mean_squared_error(ratings_test1.Rating,
                          pred_hybrid,
                          squared=True),
       np.corrcoef(ratings_test1.Rating, pred_hybrid)[0,1]))

test_cb = ratings_test1.loc[ratings_test1.UserId.isin(user_attr_train.UserId) &
                            ratings_test1.ItemId.isin(item_attr_train.ItemId)]
pred_contentbased = m_contentbased.predict(test_cb.UserId, test_cb.ItemId)
print("RMSE type 1 content-based model: %.3f [rho: %.3f]" %
      (mean_squared_error(test_cb.Rating,
                          pred_contentbased,
                          squared=True),
       np.corrcoef(test_cb.Rating, pred_contentbased)[0,1]))

RMSE type 1 non-personalized model: 0.830 [rho: 0.580]
RMSE type 1 ratings-only model: 0.808 [rho: 0.601]
RMSE type 1 hybrid model: 0.742 [rho: 0.640]
RMSE type 1 content-based model: 0.951 [rho: 0.486]


RMSE for users which were in the training data but items which were not:

In [31]:
pred_hybrid = m_collective.predict_new(ratings_test2.UserId,
                                       item_attr_test.loc[ratings_test2.ItemId])
print("RMSE type 2 hybrid model: %.3f [rho: %.3f]" %
      (mean_squared_error(ratings_test2.Rating,
                          pred_hybrid,
                          squared=True),
       np.corrcoef(ratings_test2.Rating, pred_hybrid)[0,1]))

pred_contentbased = m_contentbased.predict_new(user_attr_test.loc[ratings_test2.UserId],
                                               item_attr_test.loc[ratings_test2.ItemId])
print("RMSE type 2 content-based model: %.3f [rho: %.3f]" %
      (mean_squared_error(ratings_test2.Rating,
                          pred_contentbased,
                          squared=True),
       np.corrcoef(ratings_test2.Rating, pred_contentbased)[0,1]))

RMSE type 2 hybrid model: 1.046 [rho: 0.423]
RMSE type 2 content-based model: 0.954 [rho: 0.484]


RMSE for items which were in the training data but users which were not:

In [32]:
pred_hybrid = m_collective.predict_cold_multiple(item=ratings_test3.ItemId,
                                                 U=user_attr_test.loc[ratings_test3.UserId])
print("RMSE type 3 hybrid model: %.3f  [rho: %.3f]" %
      (mean_squared_error(ratings_test3.Rating,
                          pred_hybrid,
                          squared=True),
       np.corrcoef(ratings_test3.Rating, pred_hybrid)[0,1]))

pred_contentbased = m_contentbased.predict_new(user_attr_test.loc[ratings_test3.UserId],
                                               item_attr_test.loc[ratings_test3.ItemId])
print("RMSE type 3 content-based model: %.3f [rho: %.3f]" %
      (mean_squared_error(ratings_test3.Rating,
                          pred_contentbased,
                          squared=True),
       np.corrcoef(ratings_test3.Rating, pred_contentbased)[0,1]))

RMSE type 3 hybrid model: 0.974  [rho: 0.470]
RMSE type 3 content-based model: 0.962 [rho: 0.468]


RMSE for users and items which were not in the training data:

In [33]:
pred_contentbased = m_contentbased.predict_new(user_attr_test.loc[ratings_test4.UserId],
                                               item_attr_test.loc[ratings_test4.ItemId])
print("RMSE type 4 content-based model: %.3f [rho: %.3f]" %
      (mean_squared_error(ratings_test4.Rating,
                          pred_contentbased,
                          squared=True),
      np.corrcoef(ratings_test4.Rating, pred_contentbased)[0,1]))

RMSE type 4 content-based model: 0.973 [rho: 0.462]


<a id="p7"></a>
## 7. References

* Cortes, David. "Cold-start recommendations in Collective Matrix Factorization." arXiv preprint arXiv:1809.00366 (2018).
* Singh, Ajit P., and Geoffrey J. Gordon. "Relational learning via collective matrix factorization." Proceedings of the 14th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2008.